# Use Mistral in Azure AI and Azure ML

Use `mistralai` client to consume Mistral deployments in Azure AI and Azure ML. Notice that Mistral supports only chat completions API.

> Review the [documentation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-mistral) for the Mistral family of models at for AI Studio and for ML Studio for details on how to provision inference endpoints, regional availability, pricing and inference schema reference.

## Prerequisites

Before we start, there are certain steps we need to take to deploy the models:

* Follow the steps listed in [this](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-mistral?tabs=mistral-large#prerequisites) article to set up resources.
* Go to Azure AI Studio and select the model on Model Catalog.

    > Notice that some models may not be available in all the regions in Azure AI and Azure Machine Learning. On those cases, you can create a workspace or project in the region where the models are available and then consume it with a connection from a different one. To learn more about using connections see [Consume models with connections](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deployments-connections)

* Create a Serverless deployment using the steps listed [here](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-mistral?tabs=mistral-large#create-a-new-deployment).

Once deployed successfully, you should be assigned for an API endpoint and a security key for inference.

For more information, you should consult Azure's official documentation [here](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-mistral?tabs=mistral-large) for model deployment and inference.

To complete this tutorial, you will need to:

* Install `mistralai`:

    ```bash
    pip install mistralai
    ```

## Example

The following is an example about how to use `mistralai` with a Mistral model deployed in Azure AI and Azure ML:

In [5]:
from mistralai_azure import MistralAzure
from mistralai_azure.models import ChatCompletionRequest

To use `mistralai`, create a client and configure it as follows:

- `endpoint`: Use the endpoint URL from your deployment. Do not include either `/chat/completions` as this is included automatically by the client.
- `api_key`: Use your API key.

In [3]:
import os

endpoint = os.getenv("AZURE_MISTRAL_ENDPOINT", None)
api_key = os.getenv("AZURE_MISTRAL_API_KEY", None)

In [4]:
client = MistralAzure(azure_endpoint=endpoint, azure_api_key=api_key)

> Tip: Alternatively, you can configure your API key in the environment variables `MISTRAL_API_KEY`.

Use the client to create chat completions requests:

In [9]:
chat_response = client.chat.complete(
    messages=[
        {
            "role": "user",
            "content": "Who is the best French painter? Answer in one short sentence.",
        }
    ],
    max_tokens=50,
)

The generated text can be accessed as follows:

In [10]:
if chat_response:
    print(chat_response.choices[0].message.content)

Claude Monet.


Mistral also support the parameter `safe_prompt`. Toggling the safe prompt will prepend your messages with the following system prompt:

> Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.

In [11]:
chat_response = client.chat.complete(
    messages=[
        {
            "role": "user",
            "content": "Who is the best French painter? Answer in one short sentence.",
        }
    ],
    max_tokens=50,
    safe_prompt=True,
)

The generated text can be accessed as follows:

In [12]:
if chat_response:
    print(chat_response.choices[0].message.content)

Claude Monet is considered the best French painter.


## Example (with image inputs)

The following example shows how to handle multimodal use-cases by adding images to the model's text input. This feature is only compatible with multimodal models like `mistral-small-2503`. To run the code sample, you will need the following packages:

- `httpx`

In [13]:
import httpx
import json

url = ""  # Add the URL to your mistral-small-2503 deployment here.

headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}
payload = {
    "model": "mistral-small-2503",
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Describe this image in a short sentence."},
                {
                    "type": "image_url",
                    "image_url": {"url": "https://picsum.photos/id/237/200/300"},
                },
            ],
        }
    ],
}

resp = httpx.post(url=url, json=payload, headers=headers)
if resp:
    json_out = json.loads(resp.json()["choices"][0]["message"]["content"])
    print(json_out)

## Aditional resources

Here are some additional reference:  

* [Plan and manage costs (marketplace)](https://learn.microsoft.com/azure/ai-studio/how-to/costs-plan-manage#monitor-costs-for-models-offered-through-the-azure-marketplace)